# Assignment 2
Daniel Evans
14283316

## Imports & Read Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = 'data/train.csv'
df = pd.read_csv(data_path)
df.drop(axis=1, columns=['Unnamed: 0'], inplace=True)

In [3]:
df.head().T

,0,1,2,3,4
Name,Hyundai Creta 1.6 CRDi SX Option,Honda Jazz V,Maruti Ertiga VDI,Audi A4 New 2.0 TDI Multitronic,Nissan Micra Diesel XV
Location,Pune,Chennai,Chennai,Coimbatore,Jaipur
Year,2015,2011,2012,2013,2013
Kilometers_Driven,41000,46000,87000,40670,86999
Fuel_Type,Diesel,Petrol,Diesel,Diesel,Diesel
Transmission,Manual,Manual,Manual,Automatic,Manual
Owner_Type,First,First,First,Second,First
Mileage,19.67 kmpl,13 km/kg,20.77 kmpl,15.2 kmpl,23.08 kmpl
Engine,1582 CC,1199 CC,1248 CC,1968 CC,1461 CC
Power,126.2 bhp,88.7 bhp,88.76 bhp,140.8 bhp,63.1 bhp


# 2. Remove units
Doing this before handling missing values!

In [4]:
df.iloc[0].T

Name                 Hyundai Creta 1.6 CRDi SX Option
Location                                         Pune
Year                                             2015
Kilometers_Driven                               41000
Fuel_Type                                      Diesel
Transmission                                   Manual
Owner_Type                                      First
Mileage                                    19.67 kmpl
Engine                                        1582 CC
Power                                       126.2 bhp
Seats                                             5.0
New_Price                                         NaN
Price                                            12.5
Name: 0, dtype: object

Need to extract numerical values from 'Mileage', 'Engine', 'Power', 'New_Price'

In [5]:
df['Mileage'] = df['Mileage'].str.extract(r'(\d*) kmpl').astype(float)

In [6]:
df['Engine'] = df['Engine'].str.extract(r'(\d*) CC').astype(float)

In [7]:
df['Power'] = df['Power'].str.extract(r'(\d*) bhp').astype(float)

In [8]:
df['New_Price'] = df['New_Price'].str.extract(r'(\d*) Lakh').astype(float)

In [9]:
df.dtypes

Name                  object
Location              object
Year                   int64
Kilometers_Driven      int64
Fuel_Type             object
Transmission          object
Owner_Type            object
Mileage              float64
Engine               float64
Power                float64
Seats                float64
New_Price            float64
Price                float64
dtype: object

## 1. Missing Values

Check total number of rows  
Check total missing values for each column

In [10]:
len(df)

5847

In [11]:
df.isna().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 5
Engine                 36
Power                  36
Seats                  38
New_Price            5049
Price                   0
dtype: int64

Mileage, Engine, Power, Seats, New_Price have missing values

Column 'New_Price' is missing values for most rows (5049/5847)  
Drop 'New_Price' due to this

In [12]:
df.drop(axis=1, columns=['New_Price'], inplace=True)

See rows with missing mileage

In [13]:
df[df['Mileage'].isna()].head().T

,1,24,56,4313,4762
Name,Honda Jazz V,Nissan Micra Diesel XV,Nissan X-Trail SLX AT,Mahindra E Verito D4,Toyota Prius 2009-2016 Z4
Location,Chennai,Hyderabad,Hyderabad,Chennai,Mumbai
Year,2011,2012,2010,2016,2011
Kilometers_Driven,46000,54000,121812,50000,44000
Fuel_Type,Petrol,Diesel,Diesel,Electric,Electric
Transmission,Manual,Manual,Automatic,Automatic,Automatic
Owner_Type,First,First,First,First,First
Mileage,NaN,NaN,NaN,NaN,NaN
Engine,1199.0,1461.0,1995.0,72.0,1798.0
Power,7.0,1.0,6.0,41.0,73.0


Find the mode for Mileage for each Name  
Apply that mode to each missing Mileage value  
There will be Names for which there is no mode (only missing values for some Names)  
  
We use mode here since the mileage is declared by the manufacturer, and should be the same for every car with the same name

In [14]:
df['Mileage'] = df['Mileage'].fillna(
    df.groupby('Name')['Mileage'].transform(lambda m: m.mode().iloc[0] if not m.mode().empty else np.nan)
)
df[df['Mileage'].isna()].head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
56,Nissan X-Trail SLX AT,Hyderabad,2010,121812,Diesel,Automatic,First,NaN,1995.0,6.0,5.0,7.75
4313,Mahindra E Verito D4,Chennai,2016,50000,Electric,Automatic,First,NaN,72.0,41.0,5.0,13.00
4762,Toyota Prius 2009-2016 Z4,Mumbai,2011,44000,Electric,Automatic,First,NaN,1798.0,73.0,5.0,12.75


Drop remaining rows with missing Mileage  
Verify no more missing values for Mileage

In [15]:
df.dropna(subset=['Mileage'], inplace=True)
df[df['Mileage'].isna()] # output should be empty!!

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price


See rows with missing Engine data

In [16]:
df[df['Engine'].isna()].head().T

,186,200,709,723,1253
Name,Honda City 1.5 GXI,Maruti Swift 1.3 VXi,Maruti Swift 1.3 VXi,Land Rover Range Rover 3.0 D,Honda City 1.3 DX
Location,Ahmedabad,Kolkata,Chennai,Mumbai,Delhi
Year,2007,2010,2006,2008,2009
Kilometers_Driven,60006,42001,97800,55001,55005
Fuel_Type,Petrol,Petrol,Petrol,Diesel,Petrol
Transmission,Manual,Manual,Manual,Automatic,Manual
Owner_Type,First,First,Third,Second,First
Mileage,0.0,1.0,1.0,0.0,8.0
Engine,NaN,NaN,NaN,NaN,NaN
Power,NaN,NaN,NaN,NaN,NaN


As with Mileage, Engine volume is declared by the manufacturer and should be the same for every car with the same name  
Use the same logic for Engine as for Mileage

In [17]:
df['Engine'] = df['Engine'].fillna(
    df.groupby('Name')['Engine'].transform(lambda e: e.mode().iloc[0] if not e.mode().empty else np.nan)
)
df[df['Engine'].isna()].head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
200,Maruti Swift 1.3 VXi,Kolkata,2010,42001,Petrol,Manual,First,1.0,NaN,NaN,NaN,2.11
709,Maruti Swift 1.3 VXi,Chennai,2006,97800,Petrol,Manual,Third,1.0,NaN,NaN,NaN,1.75
723,Land Rover Range Rover 3.0 D,Mumbai,2008,55001,Diesel,Automatic,Second,0.0,NaN,NaN,NaN,26.50
1253,Honda City 1.3 DX,Delhi,2009,55005,Petrol,Manual,First,8.0,NaN,NaN,NaN,3.20
1284,Maruti Swift 1.3 ZXI,Hyderabad,2015,50295,Petrol,Manual,First,1.0,NaN,NaN,NaN,5.80


Drop remaining rows missing Engine volume data  
Verify no more missing values for Engine

In [18]:
df.dropna(subset=['Engine'], inplace=True)
df[df['Engine'].isna()] # output should be empty!!

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price


See rows with missing Power data

In [19]:
df[df['Power'].isna()].head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
186,Honda City 1.5 GXI,Ahmedabad,2007,60006,Petrol,Manual,First,0.0,1493.0,NaN,NaN,2.95
1339,Honda City 1.5 GXI,Pune,2004,115000,Petrol,Manual,Second,0.0,1493.0,NaN,NaN,1.50
2199,Toyota Etios Liva V,Pune,2012,24500,Petrol,Manual,Second,3.0,1197.0,NaN,NaN,2.95
4470,Honda Jazz Select Edition,Pune,2011,98000,Petrol,Manual,First,7.0,1198.0,NaN,NaN,3.15
5121,Honda City 1.5 GXI,Bangalore,2002,53000,Petrol,Manual,Second,0.0,1493.0,NaN,NaN,1.85


As with Mileage, Power should be a standard value for each car with the same name  
Use the same logic

In [20]:
df['Power'] = df['Power'].fillna(
    df.groupby('Name')['Power'].transform(lambda p: p.mode().iloc[0] if not p.mode().empty else np.nan)
)
df[df['Power'].isna()].head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price


Drop remaining rows where we can't infer Power

In [21]:
df.dropna(subset=['Power'], inplace=True)
df[df['Power'].isna()] # output should be empty!!

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price


Seats should follow the same pattern as well

In [22]:
df['Seats'] = df['Seats'].fillna(
    df.groupby('Name')['Seats'].transform(lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan)
)
df[df['Seats'].isna()].head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
220,Ford Figo Diesel,Bangalore,2015,70436,Diesel,Manual,First,0.0,1498.0,99.0,NaN,3.6


Drop remaining rows where we can't infer Seats

In [23]:
df.dropna(subset=['Seats'], inplace=True)
df[df['Seats'].isna()] # output should be empty!!

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price


## 3. One-Hot Encoding of Categorical Values
Fuel_Type and Transmission

In [24]:
fuel_onehot = pd.get_dummies(df['Fuel_Type'], prefix='Fuel')
df = df.join(fuel_onehot)
df.drop(columns=['Fuel_Type'], axis=1, inplace=True)

In [25]:
transmission_onehot = pd.get_dummies(df['Transmission'], prefix='Transmission')
df = df.join(transmission_onehot)
df.drop(columns=['Transmission'], axis=1, inplace=True)

In [26]:
df.head().T

,0,1,2,3,4
Name,Hyundai Creta 1.6 CRDi SX Option,Honda Jazz V,Maruti Ertiga VDI,Audi A4 New 2.0 TDI Multitronic,Nissan Micra Diesel XV
Location,Pune,Chennai,Chennai,Coimbatore,Jaipur
Year,2015,2011,2012,2013,2013
Kilometers_Driven,41000,46000,87000,40670,86999
Owner_Type,First,First,First,Second,First
Mileage,67.0,2.0,77.0,2.0,8.0
Engine,1582.0,1199.0,1248.0,1968.0,1461.0
Power,2.0,7.0,76.0,8.0,1.0
Seats,5.0,5.0,7.0,5.0,5.0
Price,12.5,4.5,6.0,17.74,3.5


## 4. Feature Engineering
Add Age, computed from current year

In [27]:
import datetime

current_year = datetime.datetime.now().year

In [28]:
df['Age'] = current_year - df['Year']

Show Name, Year, Age

In [29]:
df[['Name', 'Year', 'Age']].head()

,Name,Year,Age
0,Hyundai Creta 1.6 CRDi SX Option,2015,10
1,Honda Jazz V,2011,14
2,Maruti Ertiga VDI,2012,13
3,Audi A4 New 2.0 TDI Multitronic,2013,12
4,Nissan Micra Diesel XV,2013,12


## 5. Fundamental Operations

Select columns

In [40]:
df[['Name', 'Price', 'Age']]

,Name,Price,Age
0,Hyundai Creta 1.6 CRDi SX Option,12.50,10
1,Honda Jazz V,4.50,14
2,Maruti Ertiga VDI,6.00,13
3,Audi A4 New 2.0 TDI Multitronic,17.74,12
4,Nissan Micra Diesel XV,3.50,12
...,...,...,...
5842,Maruti Swift VDI,4.75,11
5843,Hyundai Xcent 1.1 CRDi S,4.00,10
5844,Mahindra Xylo D4 BSIV,2.90,13
5845,Maruti Wagon R VXI,2.65,12


Filter rows

In [41]:
df[df['Mileage'] > 25][['Name', 'Mileage', 'Year']]

,Name,Mileage,Year
0,Hyundai Creta 1.6 CRDi SX Option,67.0,2015
2,Maruti Ertiga VDI,77.0,2012
5,Toyota Innova Crysta 2.8 GX AT 8S,36.0,2016
6,Volkswagen Vento Diesel Comfortline,54.0,2013
8,Maruti Ciaz Zeta,56.0,2018
...,...,...,...
5825,Jaguar XF 2.2 Litre Luxury,36.0,2016
5827,Tata Bolt Revotron XT,57.0,2016
5829,Audi A6 2011-2015 2.0 TDI,68.0,2013
5836,Porsche Panamera Diesel,85.0,2013


Rename column

In [ ]:
df.rename(columns={'Mileage': 'Mileage_Metric'}, inplace=True)

In [46]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,5837,5838,5839,5840,5841,5842,5843,5844,5845,5846
Name,Hyundai Creta 1.6 CRDi SX Option,Honda Jazz V,Maruti Ertiga VDI,Audi A4 New 2.0 TDI Multitronic,Nissan Micra Diesel XV,Toyota Innova Crysta 2.8 GX AT 8S,Volkswagen Vento Diesel Comfortline,Tata Indica Vista Quadrajet LS,Maruti Ciaz Zeta,Honda City 1.5 V AT Sunroof,...,Toyota Camry Hybrid,Honda Brio 1.2 VX MT,Skoda Superb 3.6 V6 FSI,Toyota Innova 2.5 V Diesel 7-seater,Honda Amaze VX i-DTEC,Maruti Swift VDI,Hyundai Xcent 1.1 CRDi S,Mahindra Xylo D4 BSIV,Maruti Wagon R VXI,Chevrolet Beat Diesel
Location,Pune,Chennai,Chennai,Coimbatore,Jaipur,Mumbai,Pune,Chennai,Kochi,Kolkata,...,Mumbai,Delhi,Hyderabad,Coimbatore,Coimbatore,Delhi,Jaipur,Jaipur,Kolkata,Hyderabad
Year,2015,2011,2012,2013,2013,2016,2013,2012,2018,2012,...,2015,2013,2009,2011,2015,2014,2015,2012,2013,2011
Kilometers_Driven,41000,46000,87000,40670,86999,36000,64430,65932,25692,60000,...,33500,33746,53000,45004,70602,27365,100000,55000,46000,47000
Owner_Type,First,First,First,Second,First,First,First,Second,First,First,...,First,First,First,First,First,First,First,Second,First,First
Mileage_Metric,67.0,2.0,77.0,2.0,8.0,36.0,54.0,3.0,56.0,8.0,...,16.0,5.0,0.0,8.0,8.0,4.0,4.0,0.0,9.0,44.0
Engine,1582.0,1199.0,1248.0,1968.0,1461.0,2755.0,1598.0,1248.0,1462.0,1497.0,...,2494.0,1198.0,3597.0,2494.0,1498.0,1248.0,1120.0,2498.0,998.0,936.0
Power,2.0,7.0,76.0,8.0,1.0,5.0,6.0,74.0,25.0,3.0,...,2.0,8.0,6.0,102.0,6.0,74.0,71.0,112.0,1.0,6.0
Seats,5.0,5.0,7.0,5.0,5.0,8.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,7.0,5.0,5.0,5.0,8.0,5.0,5.0
Price,12.5,4.5,6.0,17.74,3.5,17.5,5.2,1.95,9.95,4.49,...,19.75,3.2,4.75,9.48,4.83,4.75,4.0,2.9,2.65,2.5


Add/modify column

In [42]:
kmpl_to_mpg = 2.35215 # coversion rate

In [47]:
df['Mileage_Imperial'] = df['Mileage_Metric'].apply(lambda m: m * kmpl_to_mpg)
df[['Name', 'Mileage_Metric', 'Mileage_Imperial']].head()

,Name,Mileage_Metric,Mileage_Imperial
0,Hyundai Creta 1.6 CRDi SX Option,67.0,157.59405
1,Honda Jazz V,2.0,4.70430
2,Maruti Ertiga VDI,77.0,181.11555
3,Audi A4 New 2.0 TDI Multitronic,2.0,4.70430
4,Nissan Micra Diesel XV,8.0,18.81720


Sort values

In [50]:
df[['Name', 'Power']].sort_values(by='Power', ascending=False)

,Name,Power
5620,Lamborghini Gallardo Coupe,560.0
5368,Bentley Continental Flying Spur,552.0
2040,Porsche Cayenne Turbo S,550.0
4942,Jaguar XJ 5.0 L V8 Supercharged,503.0
68,Porsche Cayenne 2009-2014 Turbo,500.0
...,...,...
257,Maruti A-Star Vxi,1.0
5488,Maruti Wagon R LX,1.0
2056,Maruti Baleno Delta Automatic,1.0
24,Nissan Micra Diesel XV,1.0


Describe with `groupby`

In [52]:
df.groupby('Location')['Kilometers_Driven'].mean()

Location
Ahmedabad     54974.701835
Bangalore     58157.387931
Chennai       90564.883966
Coimbatore    46906.338633
Delhi         57099.689720
Hyderabad     70608.144068
Jaipur        70657.820449
Kochi         44695.477237
Kolkata       43499.690979
Mumbai        44551.668428
Pune          69797.460616
Name: Kilometers_Driven, dtype: float64